# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
%config InlineBackend.figure_format ='retina'

#importing some useful packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2


plt.rcParams['figure.figsize'] = 16,9 

## Read in an Image

In [ ]:
#reading in an image
#image = mpimg.imread('test_images/solidWhiteRight.jpg')
image = mpimg.imread('challenge.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  

# if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, (x1, y1), (x2, y2), color, thickness):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    if lines is None:
        return
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.


# Define image properties
ysize = image.shape[0]
xsize = image.shape[1]
left_bottom = [0, ysize]
right_bottom = [xsize, ysize]
apex = [xsize/2, ysize/2]


# Define the area of interest from the original image
fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

XX, YY = np.meshgrid(np.arange(0,xsize), np.arange(0,ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) &\
                    (YY > (XX*fit_right[0] + fit_right[1])) &\
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))
        
# Check the selected region
plt.imshow(image)
x = [left_bottom[0], right_bottom[0], apex[0], left_bottom[0]]
y = [left_bottom[1], right_bottom[1], apex[1], left_bottom[1]]
plt.plot(x, y, 'b--', lw=2)
plt.title('Region of Interest')
plt.xlabel('pixels')
plt.ylabel('pixels')
plt.show()

# Find mean value of RGBs
mean_rgb = 0

for i in range(xsize):
    for j in range(ysize):
        mean_rgb = mean_rgb + sum(image[j, i])*region_thresholds[j, i]

mean_rgb=mean_rgb//(3*np.sum(region_thresholds))
print('Average RGB value in region of interest : ',mean_rgb)

# Define color selection criteria
red_threshold = 2.2*mean_rgb;
green_threshold = 2.2*mean_rgb;
blue_threshold = 2.2*mean_rgb;

print('Red Channel Threshold   \t: ', red_threshold)
print('Green Channel Threshold \t: ', green_threshold)
print('Blue Channel Threshold  \t: ', blue_threshold)

rgb_threshold = [ red_threshold, green_threshold, blue_threshold ]
color_thresholds = [ image[:,:,0] < rgb_threshold[0], image[:,:,1] < rgb_threshold[1], image[:,:,2] < rgb_threshold[2] ]

# Mask color and region
red_select = np.copy(image)
green_select = np.copy(image)
blue_select = np.copy(image)

red_select[color_thresholds[0] | ~region_thresholds ] = [0, 0, 0]
green_select[color_thresholds[1] | ~region_thresholds ] = [0, 0, 0]
blue_select[color_thresholds[2] | ~region_thresholds ] = [0, 0, 0]

red_select=red_select[:,:,0]
green_select=green_select[:,:,1]
blue_select=blue_select[:,:,2]

# Show binary images for each RGB channel

plt.imshow(red_select)
plt.title('red_channel')
plt.show()
plt.imshow(green_select)
plt.title('green_channel')
plt.show()
plt.imshow(blue_select)
plt.title('blue_channel')
plt.show()


# Convert to grayscale

gray = grayscale(image)
plt.imshow(gray, cmap='gray')
plt.title('gray_scale')
plt.show()

# Define Gaussian smoothing & Canny parameters

kernel_size = 7
blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

low_threshold = 70
high_threshold = 150
edges  = cv2.Canny(blur_gray, low_threshold, high_threshold)

edge_select=np.copy(edges)
edge_select[ ~region_thresholds ] = 0

print('GaussianBlur Kernel SIze : ', kernel_size)
print('Canny / Low Threshold : ' , low_threshold)
print('Canny / high Threshold : ', high_threshold)

plt.imshow(edge_select, cmap='Greys_r')
plt.title('Canny_edge_detection')
plt.show()


# Apply Hough Transform to find Lane Lines
# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi*1/180 # angular resolution in radians of the Hough grid
threshold = 50    # minimum number of votes (intersections in Hough grid cell)
min_line_length = ysize/4 #minimum number of pixels making up a line
max_line_gap = 200   # maximum gap in pixels between connectable line segments
line_image = np.copy(image)*0 # creating a blank to draw lines on
line_image_red = np.copy(image)*0
line_image_green = np.copy(image)*0
line_image_blue = np.copy(image)*0

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(edge_select, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)

# Iterate over the output "lines" and draw lines on a blank image
for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)

# Create a "color" binary image to combine with line image
color_edges = np.dstack((edge_select, edge_select, edge_select)) 

# Draw the lines on the edge image
lines_edges = cv2.addWeighted(color_edges, 0.8, line_image, 1, 0) 
plt.imshow(lines_edges)
plt.title('edge_detect')
plt.show()

lines_red = cv2.HoughLinesP(red_select, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
# Iterate over the output "lines" and draw lines on a blank image
for line in lines_red:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image_red,(x1,y1),(x2,y2),(255,0,0),10)

# Create a "color" binary image to combine with line image
color_edges = np.dstack((red_select, red_select, red_select)) 

# Draw the lines on the edge image
lines_edges_red = cv2.addWeighted(color_edges, 0.8, line_image_red, 1, 0) 
plt.imshow(lines_edges_red)
plt.title('red_lines')
plt.show()


lines_green = cv2.HoughLinesP(green_select, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
# Iterate over the output "lines" and draw lines on a blank image
for line in lines_green:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image_green,(x1,y1),(x2,y2),(255,0,0),10)

# Create a "color" binary image to combine with line image
color_edges = np.dstack((green_select, green_select, green_select)) 

# Draw the lines on the edge image
lines_edges_green = cv2.addWeighted(color_edges, 0.8, line_image_green, 1, 0) 
plt.imshow(lines_edges_green)
plt.title('green_lines')
plt.show()



lines_blue = cv2.HoughLinesP(blue_select, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
# Iterate over the output "lines" and draw lines on a blank image
for line in lines_blue:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image_blue,(x1,y1),(x2,y2),(255,0,0),10)

# Create a "color" binary image to combine with line image
color_edges = np.dstack((blue_select, blue_select, blue_select)) 

# Draw the lines on the edge image
lines_edges_blue = cv2.addWeighted(color_edges, 0.8, line_image_blue, 1, 0) 
plt.imshow(lines_edges_blue)
plt.title('blue_lines')
plt.show()

max_slope = 0.8*ysize/xsize

lines_left=[]
lines_right=[]
for line in lines:
    slope = (line[0][1]-line[0][3])/(line[0][0]-line[0][2])
    if abs(slope) > max_slope:
        if slope < 0:
            line_ext=np.array([math.floor((ysize*0.6-(line[0][3]-slope*line[0][2]))/slope), math.floor(ysize*0.6), math.floor((ysize-(line[0][3]-slope*line[0][2]))/slope), ysize ])
            lines_left.append([line_ext])
        else:
            line_ext=np.array([math.floor((ysize*0.6-(line[0][3]-slope*line[0][2]))/slope), math.floor(ysize*0.6), math.floor((ysize-(line[0][3]-slope*line[0][2]))/slope), ysize])
            lines_right.append([line_ext])
                

lines_left = np.asarray(lines_left)
lines_right = np.asarray(lines_right)

line_left = np.mean(lines_left, axis = 0)
line_right = np.mean(lines_right, axis = 0)

slope_left = (line_left[0][1]-line_left[0][3])/(line_left[0][0]-line_left[0][2])
slope_right = (line_right[0][1]-line_right[0][3])/(line_right[0][0]-line_right[0][2])
offset_left = math.floor((line_left[0][3]-slope_left*line_left[0][2]))
offset_right = math.floor((line_right[0][3]-slope_right*line_right[0][2]))

line_params = [ slope_left, slope_right, offset_left, offset_right]

line_selected=np.asarray([line_left.astype(int), line_right.astype(int)])

line_image_test=np.copy(image)

flag_left=0
for blueline in lines_blue:
    for x1b,y1b,x2b,y2b in blueline:
        blueslope = (y2b-y1b) / (x2b-x1b)
        bluedist = (abs(-slope_left*x1b+y1b-offset_left) + abs(-slope_left*x2b+y2b-offset_left)) / math.sqrt(slope_left**2 + 1)
        if (abs(blueslope)-abs(slope_left)<0.05):
            if bluedist < 30:
                flag_left=1
                break
    if flag_left==1:
        break
        
if flag_left==1:
    cv2.line(line_image_test, (line_selected[0][0][0], line_selected[0][0][1]), (line_selected[0][0][2], line_selected[0][0][3]), (255,0,0),10)
else:
    cv2.line(line_image_test, (line_selected[0][0][0], line_selected[0][0][1]), (line_selected[0][0][2], line_selected[0][0][3]), (255,255,0),10)

        
flag_right=0
for blueline in lines_blue:
    for x1b,y1b,x2b,y2b in blueline:
        blueslope = (y2b-y1b) / (x2b-x1b)
        bluedist = (abs(-slope_right*x1b+y1b-offset_right) + abs(-slope_right*x2b+y2b-offset_right)) / math.sqrt(slope_right**2 + 1)
        if (abs(blueslope)-abs(slope_right)<0.05):
            if bluedist < 30:
                flag_right=1
                break            
    if flag_right==1:
        break
        
if flag_right==1:
    cv2.line(line_image_test, (line_selected[1][0][0], line_selected[1][0][1]), (line_selected[1][0][2], line_selected[1][0][3]), (255,0,0),10)
else:
    cv2.line(line_image_test, (line_selected[1][0][0], line_selected[1][0][1]), (line_selected[1][0][2], line_selected[1][0][3]), (255,255,0),10)

lines_edges_test = cv2.addWeighted(color_edges, 0.8, line_image_test, 1, 0) 
plt.imshow(lines_edges_test)
plt.show()

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    
    # TODO: Build your pipeline that will draw lane lines on the test_images
    # then save them to the test_images directory.


    # Define image properties
    
    ysize = image.shape[0]
    xsize = image.shape[1]
    left_bottom = [0, ysize]
    right_bottom = [xsize, ysize]
    apex = [xsize/2, ysize/2]


    # Define the area of interest from the original image
    fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
    fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

    XX, YY = np.meshgrid(np.arange(0,xsize), np.arange(0,ysize))
    region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) &\
                        (YY > (XX*fit_right[0] + fit_right[1])) &\
                        (YY < (XX*fit_bottom[0] + fit_bottom[1]))
            
    # Check the selected region
    plt.imshow(image)
    x = [left_bottom[0], right_bottom[0], apex[0], left_bottom[0]]
    y = [left_bottom[1], right_bottom[1], apex[1], left_bottom[1]]
    # Find mean value of RGBs
    mean_rgb = 0

    for i in range(xsize):
        for j in range(ysize):
            mean_rgb = mean_rgb + sum(image[j, i])*region_thresholds[j, i]

    mean_rgb=mean_rgb//(3*np.sum(region_thresholds))

    # Define color selection criteria
    red_threshold = 2.2*mean_rgb;
    green_threshold = 2.2*mean_rgb;
    blue_threshold = 2.2*mean_rgb;

    rgb_threshold = [ red_threshold, green_threshold, blue_threshold ]
    color_thresholds = [ image[:,:,0] < rgb_threshold[0], image[:,:,1] < rgb_threshold[1], image[:,:,2] < rgb_threshold[2] ]

    # Mask color and region
    red_select = np.copy(image)
    green_select = np.copy(image)
    blue_select = np.copy(image)

    red_select[color_thresholds[0] | ~region_thresholds ] = [0, 0, 0]
    green_select[color_thresholds[1] | ~region_thresholds ] = [0, 0, 0]
    blue_select[color_thresholds[2] | ~region_thresholds ] = [0, 0, 0]

    red_select=red_select[:,:,0]
    green_select=green_select[:,:,1]
    blue_select=blue_select[:,:,2]



    # Convert to grayscale

    gray = grayscale(image)

    # Define Gaussian smoothing & Canny parameters

    kernel_size = 7
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    low_threshold = 70
    high_threshold = 150
    edges  = cv2.Canny(blur_gray, low_threshold, high_threshold)

    edge_select=np.copy(edges)
    edge_select[ ~region_thresholds ] = 0

    # Apply Hough Transform to find Lane Lines
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi*1/180 # angular resolution in radians of the Hough grid
    threshold = 30    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = ysize/5 #minimum number of pixels making up a line
    max_line_gap = 200   # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    line_image_red = np.copy(image)*0
    line_image_green = np.copy(image)*0
    line_image_blue = np.copy(image)*0

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edge_select, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edge_select, edge_select, edge_select)) 

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(color_edges, 0.8, line_image, 1, 0) 


    lines_red = cv2.HoughLinesP(red_select, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines_red:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image_red,(x1,y1),(x2,y2),(255,0,0),10)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((red_select, red_select, red_select)) 

    # Draw the lines on the edge image
    lines_edges_red = cv2.addWeighted(color_edges, 0.8, line_image_red, 1, 0) 

    lines_green = cv2.HoughLinesP(green_select, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines_green:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image_green,(x1,y1),(x2,y2),(255,0,0),10)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((green_select, green_select, green_select)) 

    # Draw the lines on the edge image
    lines_edges_green = cv2.addWeighted(color_edges, 0.8, line_image_green, 1, 0) 

    lines_blue = cv2.HoughLinesP(blue_select, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines_blue:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image_blue,(x1,y1),(x2,y2),(255,0,0),10)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((blue_select, blue_select, blue_select)) 

    # Draw the lines on the edge image
    lines_edges_blue = cv2.addWeighted(color_edges, 0.8, line_image_blue, 1, 0) 

    max_slope = 0.8*ysize/xsize

    lines_left=[]
    lines_right=[]
    for line in lines:
        slope = (line[0][1]-line[0][3])/(line[0][0]-line[0][2])
        if abs(slope) > max_slope:
            if slope < 0:
                line_ext=np.array([math.floor((ysize*0.6-(line[0][3]-slope*line[0][2]))/slope), math.floor(ysize*0.6), math.floor((ysize-(line[0][3]-slope*line[0][2]))/slope), ysize ])
                lines_left.append([line_ext])
            else:
                line_ext=np.array([math.floor((ysize*0.6-(line[0][3]-slope*line[0][2]))/slope), math.floor(ysize*0.6), math.floor((ysize-(line[0][3]-slope*line[0][2]))/slope), ysize])
                lines_right.append([line_ext])
                    

    lines_left = np.asarray(lines_left)
    lines_right = np.asarray(lines_right)

    line_left = np.mean(lines_left, axis = 0)
    line_right = np.mean(lines_right, axis = 0)

    slope_left = (line_left[0][1]-line_left[0][3])/(line_left[0][0]-line_left[0][2])
    slope_right = (line_right[0][1]-line_right[0][3])/(line_right[0][0]-line_right[0][2])
    offset_left = math.floor((line_left[0][3]-slope_left*line_left[0][2]))
    offset_right = math.floor((line_right[0][3]-slope_right*line_right[0][2]))

    line_params = [ slope_left, slope_right, offset_left, offset_right]

    line_selected=np.asarray([line_left.astype(int), line_right.astype(int)])

    line_image_test=np.copy(image)

    flag_left=0
    for blueline in lines_blue:
        for x1b,y1b,x2b,y2b in blueline:
            blueslope = (y2b-y1b) / (x2b-x1b)
            bluedist = (abs(-slope_left*x1b+y1b-offset_left) + abs(-slope_left*x2b+y2b-offset_left)) / math.sqrt(slope_left**2 + 1)
            if (abs(blueslope)-abs(slope_left)<0.05):
                if bluedist < 30:
                    flag_left=1
                    break
        if flag_left==1:
            break
            
    if flag_left==1:
        cv2.line(line_image_test, (line_selected[0][0][0], line_selected[0][0][1]), (line_selected[0][0][2], line_selected[0][0][3]), (255,0,0),10)
    else:
        cv2.line(line_image_test, (line_selected[0][0][0], line_selected[0][0][1]), (line_selected[0][0][2], line_selected[0][0][3]), (255,255,0),10)

            
    flag_right=0
    for blueline in lines_blue:
        for x1b,y1b,x2b,y2b in blueline:
            blueslope = (y2b-y1b) / (x2b-x1b)
            bluedist = (abs(-slope_right*x1b+y1b-offset_right) + abs(-slope_right*x2b+y2b-offset_right)) / math.sqrt(slope_right**2 + 1)
            if (abs(blueslope)-abs(slope_right)<0.05):
                if bluedist < 30:
                    flag_right=1
                    break            
        if flag_right==1:
            break
            
    if flag_right==1:
        cv2.line(line_image_test, (line_selected[1][0][0], line_selected[1][0][1]), (line_selected[1][0][2], line_selected[1][0][3]), (255,0,0),10)
    else:
        cv2.line(line_image_test, (line_selected[1][0][0], line_selected[1][0][1]), (line_selected[1][0][2], line_selected[1][0][3]), (255,255,0),10)

    lines_edges_test = cv2.addWeighted(color_edges, 0.8, line_image_test, 1, 0) 
    plt.imshow(lines_edges_test)
    plt.show()


    return lines_edges_test

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
%matplotlib notebook
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))